# Example Operational Forecast

The sample shown is Typhoon Kompasu (2021), date 10 October 2021. In the end, there was no T1, but a T3 and a T8 in by 12 October.

System requirements:
- Two Anaconda (recommended) environments, one for dynamical data preprocessing (Python 2 + PyNIO) and another for the models (Python 3)  
  - This env requires numpy, pandas, scipy, pygam, sktime, xgboost, sklearn, geopy, geographiclib.
- At least 2.5GB of free disk space to hold the models, extra 500MB recommended to hold dynamical data
- 4+ CPU cores and preferably 8+ GB RAM (when the models are loaded, around 2.5GB is used), also a GPU may be needed for xgboost

Steps:
1. Make sure this is the correct environment - running Python 3 and has numpy, pandas, scipy, pygam, sktime, xgboost and sklearn available   
2. Make sure you have preprocessed the correct dynamical data and placed them under `./data` folder  
3. Execute the following cells until you encounter markdown  

In [1]:
# import statements
import pandas as pd
import numpy as np
import joblib
import sys
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

Step 4: Enter model paths and execute the cells

In [2]:
# The folder housing all the models we will need
model_folder = "./../models/"

# The exact file names of the ensemble voting estimator, folder excluded
ensemble_name = 'ensemble_7member_ridge_2022-04-17 20-14.skl'

# The exact file names of the ensemble member models, folder excluded
xgb_clf_name = 'experimental_model_gscv_xgb_clf_2022-04-12 19-13.skl'
xgb_regr_name = 'experimental_model_gscv_xgb_regr_2022-04-16 20-15.skl'
extra_trees_clf_name = 'experimental_model_gscv_extra_trees_clf_2022-04-12 19-18.skl'
extra_trees_regr_name = 'experimental_model_gscv_extra_trees_regr_2022-04-16 17-15.skl'
mlp_clf_name = 'experimental_model_mlpclf_2022-04-12 21-18.skl'
tsfs_clf_name = 'experimental_model_calibrated_gscv_tsfs_clf_2022-04-12 20-11.skl'
gam_name = 'experimental_model_gam_tsnv_2022-04-14 18-20.pkl'

In [3]:
# load models
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
import xgboost as xgb
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.neural_network import MLPClassifier
import pickle as pk
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, mutual_info_classif #, SelectFromModel
from functools import partial
from pygam import LinearGAM, PoissonGAM
from sklearn.linear_model import Ridge

xgb_clf = joblib.load(model_folder + xgb_clf_name)
xgb_regr = joblib.load(model_folder + xgb_regr_name)
extra_trees_gscv_clf = joblib.load(model_folder + extra_trees_clf_name)
extra_trees_gscv_regr = joblib.load(model_folder + extra_trees_regr_name)
mlp_clf = joblib.load(model_folder + mlp_clf_name)
tsfs_calibrated_clf = joblib.load(model_folder + tsfs_clf_name)

file = open(model_folder + gam_name, "rb") 
gam = pk.load(file)
file.close()

file = open(model_folder + ensemble_name, "rb")
ensembles = pk.load(file)
file.close()

warnings.simplefilter(action='ignore', category=FutureWarning)

Step 5: Enter TC data particulars

Much of the information you'll need can be sourced from [RAMMB](https://rammb-data.cira.colostate.edu/tc_realtime/).

In [4]:
# Current date and hour (preferably at 00, 06, 12 or 18)
YY00, MM00, DD00, HH00 = 2021, 10, 11, 18 

# HKO warning signal status over the last 24 hours
# MI: true if T1, LI: true if T3, SI: true if T8 or plus
# DS: true if TC in HKO 100 radius
MI_STATUS00, LI_STATUS00, SI_STATUS00, DS_STATUS00 = False, False, False, False
MI_STATUS06, LI_STATUS06, SI_STATUS06, DS_STATUS06 = False, False, False, False
MI_STATUS12, LI_STATUS12, SI_STATUS12, DS_STATUS12 = False, False, False, False
MI_STATUS18, LI_STATUS18, SI_STATUS18, DS_STATUS18 = False, False, False, False
MI_STATUS24, LI_STATUS24, SI_STATUS24, DS_STATUS24 = False, False, False, False

# TC position and intensity over last 24 hours
LAT00, LON00, VMAX00 = 18.8, 120.5, 55
LAT06, LON06, VMAX06 = 18.9, 121.6, 55
LAT12, LON12, VMAX12 = 18.9, 123.1, 50
LAT18, LON18, VMAX18 = 18.5, 124.2, 50
LAT24, LON24, VMAX24 = 18.8, 124.9, 45
LAT30, LON30 = 18.4, 126.6 # needed to estimate speed and heading

Step 6: Load dynamical variables values

In [5]:
import csv

def read_csv_to_dict(filename):
    '''Takes in a filename to a CSV file (without extension) and returns its content as a dictionary (str -> float)'''
    with open(filename, 'r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:float(rows[1]) for rows in reader}
    return mydict

easm = read_csv_to_dict("./easm_indices.csv")
hi_humid = read_csv_to_dict("./hi_humid.csv")
hk_u_winds = read_csv_to_dict("./hk_u_winds.csv")
lo_humid = read_csv_to_dict("./lo_humid.csv")
hk_v_winds = read_csv_to_dict("./hk_v_winds.csv")
mlvws = read_csv_to_dict("./mlvws.csv") 
pott = read_csv_to_dict("./pott.csv")
temp_surface = read_csv_to_dict("./temp_surface.csv")
temp200 = read_csv_to_dict("./temp200.csv")
u200 = read_csv_to_dict("./u200.csv")
u500 = read_csv_to_dict("./u500.csv")
ulvws = read_csv_to_dict("./ulvws.csv")
v500 = read_csv_to_dict("./v500.csv")
vort850 = read_csv_to_dict("./vort850.csv")
westerly = read_csv_to_dict("./westerly_indices.csv")
wnpsh_area = read_csv_to_dict("./wnpsh_area_indices.csv")
wnpsh_intensity = read_csv_to_dict("./wnpsh_intensity_indices.csv")
wnpsh_extension = read_csv_to_dict("./wnpsh_extension_indices.csv")

Step 7: Finish input data pre-processing, execute the cells below

In [49]:
# converting to arrays to ease processing
# the format before was for easier fill in
mi = [MI_STATUS00,MI_STATUS06,MI_STATUS12,MI_STATUS18,MI_STATUS24]
li = [LI_STATUS00,LI_STATUS06,LI_STATUS12,LI_STATUS18,LI_STATUS24]
si = [SI_STATUS00,SI_STATUS06,SI_STATUS12,SI_STATUS18,SI_STATUS24]
ds = [DS_STATUS00,DS_STATUS06,DS_STATUS12,DS_STATUS18,DS_STATUS24]
lat = [LAT00,LAT06,LAT12,LAT18,LAT24,LAT30]
lon = [LON00,LON06,LON12,LON18,LON24,LON30]
vmax = [VMAX00,VMAX06,VMAX12,VMAX18,VMAX24]

# helper functions
from geographiclib.geodesic import Geodesic
import geopy.distance

def find_azimuth(lat0: float, long0: float, lat1: float, long1: float):
    '''Takes in two pairs of latitudes and longitudes, returns latter's forward azimuth from former.'''
    azimuth = Geodesic.WGS84.Inverse(lat0, long0, lat1, long1)['azi1']
    # the given azimuth is in [-180, 180], I prefer [0, 360] instead
    return (azimuth + 360.0) % 360.0

def azimuth_to_HK(lat: float, long: float):
    '''Takes in a latitude and a longitude, returns its forward azimuth from Hong Kong.'''
    # constants
    hko_lat = 22.302219
    hko_long = 114.174637
    azimuth = find_azimuth(hko_lat, hko_long, lat, long)
    return azimuth

def estimate_speed(lat0, long0, lat1, long1):
    '''Takes in two pairs of latitudes and longitudes, returns the speed (knots) needed to cover the distance between them in 6 hours.'''
    distance_travelled = geopy.distance.distance((lat0, long0), (lat1, long1)).km * 1.852
    return distance_travelled / 6

# columns of the data sample
columns = []
for i in range(0, 24+6, 6):
    columns.append('MM{0:02d}'.format(i))
    columns.append('DD{0:02d}'.format(i))

    columns.append('MI_STATUS{0:02d}'.format(i))
    columns.append('LI_STATUS{0:02d}'.format(i))
    columns.append('SI_STATUS{0:02d}'.format(i))
    columns.append('DS_STATUS{0:02d}'.format(i))

    columns.append('DIST{0:02d}'.format(i))
    columns.append('AZM{0:02d}'.format(i))
    columns.append('SPEED{0:02d}'.format(i))
    columns.append('DIR{0:02d}'.format(i))
    columns.append('VMAX{0:02d}'.format(i))

    if i != 24:
        columns.append('DVMAX{0:02d}'.format(i))
        
    columns.append('ULVWS{0:02d}'.format(i))
    columns.append('MLVWS{0:02d}'.format(i))
    
    columns.append('HI_HUMID{0:02d}'.format(i))
    columns.append('LO_HUMID{0:02d}'.format(i))
    
    columns.append('STEMP{0:02d}'.format(i))
    columns.append('UTEMP{0:02d}'.format(i))
    
    columns.append('U_HK{0:02d}'.format(i))
    columns.append('V_HK{0:02d}'.format(i))    
    columns.append('U200{0:02d}'.format(i))
    columns.append('U500{0:02d}'.format(i))
    columns.append('V500{0:02d}'.format(i))    
    columns.append('EASM{0:02d}'.format(i))
    
    columns.append('VORT{0:02d}'.format(i))
    
    columns.append('WESTERLY{0:02d}'.format(i))
    columns.append('SH_AREA{0:02d}'.format(i))
    columns.append('SH_INT{0:02d}'.format(i))
    columns.append('SH_EXT{0:02d}'.format(i))
    
    columns.append('POTT{0:02d}'.format(i))    
print(len(columns)) # this should be 149

149


In [50]:
# processing
from datetime import datetime, timedelta
from numpy import dtype

data_X = []

time_stamp = datetime(YY00, MM00, DD00, HH00)
for i in range(5):
    # time info
    data_X.append(time_stamp.month)
    data_X.append(time_stamp.day)
    
    # endogenous variables
    data_X.append(mi[i])
    data_X.append(li[i])
    data_X.append(si[i])
    data_X.append(ds[i])
    
    # calculate distance, azimuth, speed and direction
    temp = geopy.distance.distance((22.302219, 114.174637), (lat[i], lon[i]))
    data_X.append(temp.km) # dist
    data_X.append(azimuth_to_HK(lat[i], lon[i])) # azm
    data_X.append(estimate_speed(lat[i+1], lon[i+1], lat[i], lon[i])) # speed
    data_X.append(find_azimuth(lat[i+1], lon[i+1], lat[i], lon[i])) # dir
    
    # vmax and dvmax
    data_X.append(vmax[i])
    if i != 4:
        data_X.append(vmax[i] - vmax[i+1])
    
    # dynamical variables
    key = time_stamp.strftime("%Y%m%d_%H_%M")
    
    data_X.append(ulvws[key])
    data_X.append(mlvws[key])
    data_X.append(hi_humid[key])
    data_X.append(lo_humid[key])
    data_X.append(temp_surface[key])
    data_X.append(temp200[key])
    data_X.append(hk_u_winds[key])
    data_X.append(hk_v_winds[key])
    data_X.append(u200[key])
    data_X.append(u500[key])
    data_X.append(v500[key])
    data_X.append(easm[key])
    data_X.append(vort850[key] * 1e6)
    data_X.append(westerly[key])
    data_X.append(int(wnpsh_area[key]))
    data_X.append(wnpsh_intensity[key])
    data_X.append(int(wnpsh_extension[key]))
    data_X.append(pott[key])
    
    time_stamp -= timedelta(hours=6)
    
data_X = pd.DataFrame(data_X)    
print("This number should be 149:", len(data_X))

# convert to 1 sample, 149 cols
data_X = data_X.transpose()

# fix types, else XGBoost will complain
types = [
    dtype('int32'), dtype('int32'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('bool'), 
    dtype('float64'), dtype('float64'), dtype('int32'), dtype('int32'), dtype('int32'), 
    dtype('int32'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'),
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('int64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('int32'), 
    dtype('int32'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('bool'), 
    dtype('float64'), dtype('float64'), dtype('int32'), dtype('int32'), dtype('int32'), 
    dtype('int32'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('int64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('int32'), 
    dtype('int32'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('float64'),
    dtype('float64'), dtype('int32'), dtype('int32'), dtype('int32'), dtype('int32'), dtype('float64'),
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'),
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')
    , dtype('float64'), dtype('int64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('int32'), 
    dtype('int32'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('bool'), dtype('float64'), dtype('float64'), 
    dtype('int32'), dtype('int32'), dtype('int32'), dtype('int32'), dtype('float64'), dtype('float64'), 
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), 
    dtype('int64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('int32'), dtype('int32'), dtype('bool'),
    dtype('bool'), dtype('bool'), dtype('bool'), dtype('float64'), dtype('float64'), dtype('int32'), dtype('int32'),
    dtype('int32'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'),
    dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'),
    dtype('float64'), dtype('float64'), dtype('int64'), dtype('float64'), dtype('int64'), dtype('float64')
]
data_X = data_X.astype(dict(zip(range(len(types)), types)))

data_X = data_X.set_axis(columns, axis=1, inplace=False)
print("Input data:")
print(data_X)

This number should be 149: 149
Input data:
   MM00  DD00  MI_STATUS00  LI_STATUS00  SI_STATUS00  DS_STATUS00      DIST00  \
0    10    11        False        False        False        False  764.957672   

        AZM00  SPEED00  DIR00  ...    U20024    U50024    V50024     EASM24  \
0  119.307465       35    264  ...  8.959722  8.959722  1.929861  43.924152   

      VORT24  WESTERLY24  SH_AREA24  SH_INT24  SH_EXT24     POTT24  
0  37.749996   469.44098          0       0.0         0  296.97998  

[1 rows x 149 columns]


In [51]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Columns: 149 entries, MM00 to POTT24
dtypes: bool(20), float64(90), int32(29), int64(10)
memory usage: 1.0 KB


In [61]:
# for sktime
from sktime.transformations.panel.compose import ColumnConcatenator

def convert_X(dataset_X):
    '''Takes in a (n_samples, n_features) Pandas dataframe and returns it in shape (n_samples, n_features, time_series_length)'''
    to_drop = ["DVMAX{0:02d}".format(i) for i in range(0, 24, 6)]
    print("dropping", to_drop)
    dataset_X = dataset_X.drop(to_drop, axis=1)
    
    processed_samples = 0
    new_dataset = []
    for index, row in dataset_X.iterrows():
        new_row = []

        # obtain time series for each feature        
        for i in range(29):
            feature_name = dataset_X.columns[i][:-2]
            feature_series = []
            for j in range(0, 24+6, 6):        
                feature_series.append(row.loc["{0}{1:02d}".format(feature_name, j)]) # access by column name
            feature_series.reverse() # newest data come last
            feature_series = pd.Series(data=feature_series) # correct type for each cell
            new_row.append(feature_series)

        # new_row = pd.Series(data=new_row, index=new_features)
        new_dataset.append(new_row)
        processed_samples += 1

        if processed_samples % 1000 == 0:
            print("Finished concatenating {0}/{1} samples...".format(processed_samples, dataset_X.shape[0]))
            
    # convert types back
    # converted_X = pd.DataFrame(new_dataset, columns=new_features)
    converted_X = np.array(new_dataset)
    print("Completed")
    return converted_X
    
concat_data_X = convert_X(data_X)
print("Before transform:", concat_data_X.shape)
concat_data_X = ColumnConcatenator().fit_transform(concat_data_X)
print("After transform:", concat_data_X.shape)
print("Each element is:", concat_data_X.iloc[0].iloc[0].shape)

dropping ['DVMAX00', 'DVMAX06', 'DVMAX12', 'DVMAX18']
Completed
Before transform: (1, 29, 5)
After transform: (1, 1)
Each element is: (145,)


Step 8: The ensemble members generate their predictions

In [63]:
all_input_preds = []

for i in range(4):
    train_preds = []

    proba = np.array(xgb_clf.predict_proba(data_X))[i,:,1]
    train_preds.append(proba)
    proba = np.array(xgb_regr.predict(data_X))[:,i]
    train_preds.append(proba)
    
    proba = np.array(extra_trees_gscv_clf.predict_proba(data_X))[i,:,1]
    train_preds.append(proba)
    proba = np.array(extra_trees_gscv_regr.predict(data_X))[:,i]
    train_preds.append(proba)
    
    train_preds.append(mlp_clf.predict_proba(data_X)[:,i])
    
    proba = np.array(tsfs_calibrated_clf.predict_proba(concat_data_X))[i,:,1]
    train_preds.append(proba)
    
    in_data_X = gam["poly"][i].transform(data_X.iloc[:,:(gam["input_feature_count"][i])])
    tr_data_X = gam["fs"][i].transform(in_data_X)
    preds = np.clip(gam["gam"][i].predict(tr_data_X), 0, 1)
    train_preds.append(preds)

    train_preds = np.array(train_preds).T
    print("Predictand {0} finished, predictions shape should look like (1,7): {1}".format(i, train_preds.shape))
    all_input_preds.append(train_preds)
    
all_input_preds = np.array(all_input_preds)
print("All done! Prediction shapes should look like (4,1,7):", all_input_preds.shape)

/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgb

Predictand 0 finished, predictions shape should look like (1,7): (1, 7)


/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgb

Predictand 1 finished, predictions shape should look like (1,7): (1, 7)


/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgb

Predictand 2 finished, predictions shape should look like (1,7): (1, 7)


/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
/userhome/cs/u3556490/anaconda3/envs/test/lib/python3.9/site-packages/xgb

Predictand 3 finished, predictions shape should look like (1,7): (1, 7)
All done! Prediction shapes should look like (4,1,7): (4, 1, 7)


Step 9: Ensemble makes final predictions

In [69]:
all_preds = []

for i in range(4):
    test_preds = ensembles[i].predict(all_input_preds[i,:,:])
    print("The shape should be (1,):", test_preds.shape)
    all_preds.append(float(test_preds))
    
all_preds = np.clip(all_preds, 0, 1)
print("Raw probabilities:", all_preds)

The shape should be (1,): (1,)
The shape should be (1,): (1,)
The shape should be (1,): (1,)
The shape should be (1,): (1,)
Raw probabilities: [0.58010094 0.19066854 0.05907501 0.01526025]


Step 10: Look at the final report

In [73]:
decision_thresholds = [0.40494, 0.37428, 0.39512, 0.27891]

# show probabilities
print("Tropical cyclone impact level probability forecast")
print("Date/time of forecast: UTC {0:04d}/{1:02d}/{2:02d} {3:02d}:00".format(YY00,MM00,DD00,HH00))
print("-----------------------------------------------------------------")
print("Probability of minimal impact (T1): {0:.3%}".format(all_preds[0]))
print("Probability of limited impact (T3): {0:.3%}".format(all_preds[1]))
print("Probability of substantial impact (T8 to T10): {0:.3%}".format(all_preds[2]))
print("Probability of direct strike (100 km radius of HKO): {0:.3%}".format(all_preds[3]))
print("-----------------------------------------------------------------")
print("The above forecast is valid for 72 hours.")
print("If any of the four probabilities are bigger than {0:.2%}, {1:.2%}, {2:.2%} and {3:.2%} respectively,".format(
    decision_thresholds[0], decision_thresholds[1], decision_thresholds[2], decision_thresholds[3]))
print("then you can assume that the corresponding is more likely to happen than not.")
print("These figures are not official. Please consult the HKO for more reliable forecasts.")

Tropical cyclone impact level probability forecast
Date/time of forecast: UTC 2021/10/11 18:00
-----------------------------------------------------------------
Probability of minimal impact (T1): 58.010%
Probability of limited impact (T3): 19.067%
Probability of substantial impact (T8 to T10): 5.908%
Probability of direct strike (100 km radius of HKO): 1.526%
-----------------------------------------------------------------
The above forecast is valid for 72 hours.
If any of the four probabilities are bigger than 40.49%, 37.43%, 39.51% and 27.89% respectively,
then you can assume that the corresponding is more likely to happen than not.
These figures are not official. Please consult the HKO for more reliable forecasts.
